# 04-rag-with-milvus

Milvus と LangChain を用いて簡易的な RAG 構成を実現します。

In [ ]:
import os
from dotenv import load_dotenv
import uuid

from langchain_cohere.chat_models import ChatCohere
from langchain_cohere.embeddings import CohereEmbeddings
from langchain_milvus import Milvus
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langfuse import Langfuse
from langfuse.callback import CallbackHandler

In [ ]:
_ = load_dotenv()

# Cohere
cohere_api_key = os.getenv("COHERE_API_KEY")
# Langfuse
secret_key = os.getenv("LANGFUSE_SECRET_KEY")
public_key = os.getenv("LANGFUSE_PUBLIC_KEY")
langfuse_host = os.getenv("LANGFUSE_HOST")
# Milvus
uri = os.getenv("MILVUS_URI")
collection_name = os.getenv("COLLECTION_NAME")

embed = CohereEmbeddings(cohere_api_key=cohere_api_key, model="embed-multilingual-v3.0")
milvus = Milvus(
    embedding_function=embed,
    collection_name=collection_name,
    connection_args={"uri": uri}
)
langfuse = Langfuse(
    secret_key=secret_key,
    public_key=public_key,
    host=langfuse_host
)
langfuse_handler = CallbackHandler(
    secret_key=secret_key,
    public_key=public_key,
    host=langfuse_host
)

preamble = langfuse.get_prompt(name="demo-preamble", type="text")

chat = ChatCohere(
    cohere_api_key=cohere_api_key,
    max_tokens=500,
    temperature=0.3,
    k=0,
    p=0.75,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    preamble=preamble.prompt
)

In [ ]:
prompt_template = PromptTemplate.from_template(
    langfuse.get_prompt(name="demo-user-prompt", type="text").prompt,
    template_format="jinja2"
)

retriever = milvus.as_retriever()

chain = (
    {"question": RunnablePassthrough(), "context": retriever}
    | prompt_template
    | chat
    | StrOutputParser()
)

response = chain.stream(
    "おちゃかふぇってなんですか？",
    config={"callbacks": [langfuse_handler], "configurable": {"session_id": str(uuid.uuid4())}},
)

for chunk in response:
    print(chunk, end="")